In [ ]:
#!share/d.deyneko/metric_forecaster/metrics_forecaster.ipynb

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima_process import ArmaProcess
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
import statsmodels.api as sm
from scipy.stats import ttest_1samp
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objs as go
import digger as dg
from scipy.stats import pearsonr
import sqlalchemy
import plotly.graph_objects as go
from scipy import stats
import os
import json
from sqlalchemy import create_engine
from clickhouse_sqlalchemy import make_session
from sqlalchemy.sql import text
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import timedelta, datetime, date
from prophet import Prophet
from IPython.display import HTML, display, Markdown, clear_output
from ipywidgets import widgets, VBox, HBox, Output
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from clickhouse_driver import Client


connection = {
    'host': 'http://click.c.test:8123',
    'password': '123123',
    'user': 'default',
    'database': 'metrics'
}


FRACTION_METRICS = {
    "DAU_with_friend_ratio_7d",
    "ratio_3plus_friends"
}